In [1]:
import time
import json
import sys
import os
import glob
import re
import logging
import traceback
import csv
import datetime

from collections import deque

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client,
                                    create_flows_client)
from funcx.sdk.client import FuncXClient
from funcx.serialize import FuncXSerializer

# Flow Service Client ID
CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [ ]:
# Globus Online Endpoints
src_endpoint = 'dd916908-0072-11e7-badc-22000b9a448b' # mona4 #'hostel' #aps/workstation
dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta'

# FuncX endpoint at ThetaGPU (ALCF) and Prisma (APS)
theta_fx_endpoint = '5b5de98e-7701-4484-b03c-619a557c5fe6' # theta-ptycho-8w-1n-2cn #'2ab22e1f-4cf1-47e9-a40c-dd8c58b41d73' #theta-ptycho-8w-1n
prisma_fx_endpoint = '37766683-85e2-49df-a1a7-6a070f566022' #mona4 (prisma-ptycho)

In [ ]:
fxc = FuncXClient()

In [ ]:
# FuncX function definition for remote ThetaGPU reconstruction call

def ptycho(**data):
    """Test the ptycho tool"""
    import os
    import subprocess
    import logging
    from subprocess import PIPE

    rec_ngpu = data['rec_ngpu']
    wid = data['wid']
    dataset_name = data['dataset_name']

    log_file_name = "/grand/hp-ptycho/bicer/20210723_workflow-Xu/logs/funcx-ptycho-{}-w{}-g{}.log".format(dataset_name, wid, rec_ngpu)
    logging.basicConfig(filename=log_file_name,
                        filemode='a',
                        format='%(asctime)s %(levelname)s %(message)s',
                        level=logging.INFO,
                        datefmt='%Y-%m-%d %H:%M:%S')

    logging.info("Starting ptycho funcx function.")

    remote_log_file_name = "/grand/hp-ptycho/bicer/20210723_workflow-Xu/logs/tike-wf-{}-w{}-g{}.log".format(dataset_name, wid, rec_ngpu)

    python_path = data['python_path']
    script_path = data['script_path']

    #recon. script parameters
    ifpath = data['ifpath']
    pospath = data['position_path']
    probepath = data['probe_path']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    rec_overwrite = '--overwrite' if (('rec_overwrite' in data) and data['rec_overwrite']) else ''
    rec_auto_pin = '--auto-pin' if (('rec_auto_pin' in data) and data['rec_auto_pin']) else ''
    rec_gpu_id = data['rec_gpu_id']
    rec_log_filename = remote_log_file_name

    try:
        os.mkdir(ofpath)
    except:
        pass

    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' --position-path={pospath} --probe-path={probepath} {rec_overwrite} {rec_auto_pin} --gpu-id={rec_gpu_id} --folder='{ofpath}' --log-file='{rec_log_filename}'"
    logging.info(f"Running command: {cmd}")

    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr

func_ptycho_uuid = fxc.register_function(ptycho)
print(func_ptycho_uuid)

In [ ]:
# FuncX function definition for remote DAQ machine directory calls
def get_folder_paths(path):
    import glob
    import re

    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join(reversed(v)).index('/'):]).group(0)))


def get_file_paths(path):
    import glob
    import re

    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join((reversed(v))).index('/'):v.index('.')]).group(0)))

fx_func_get_file_paths_uuid = fxc.register_function(get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

In [ ]:
# Local helper functions for keeping track of the active scans
def append_activated_iids(data, csv_path='/home/beams/TBICER/logs/activated_iids.csv'):
    print("adding row to activated: {}".format(data))
    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        for row in data:
            writer.writerow([row, datetime.datetime.now()])
            
def read_activated_iids(csv_path='/home/beams/TBICER/logs/activated_iids.csv'):
    vals = []
    with open(csv_path, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            vals.append(row)
    return vals

In [ ]:
# Globus Automate flow definition (Gladier will cleanup this)

flow_definition = {
  "Comment": "Ptychographic reconstruction workflow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint",
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": True
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 14400,
      "Next": "PosTransfer"
    },
    "PosTransfer": {
      "Comment": "Transfer position file",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint",
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_pos_path",
            "destination_path.$": "$.input.dest_pos_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 14400,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      #"ActionUrl": "https://api.funcx.org/automate",
        "ActionUrl": "https://automate.funcx.org",
      #"ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
        "ActionScope": "https://auth.globus.org/scopes/b3db7e59-a6f1-4947-95c2-59d6b7a70f8c/action_all",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "function.$": "$.input.fx_id",
            "payload.$": "$.input.params"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 14400,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint",
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.result_path",
            "destination_path.$": "$.input.source_result_path",
            "recursive": True #False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 14400,
      "End": True
    },
  }
}

In [ ]:
### Create a flow with the above flow definition ###
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, validate_schema=False, title="Simple ptycho data analysis flow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

In [ ]:
dataset_name = "battery"
q1 = deque()
log_file_name = "/home/beams/TBICER/logs/20210723_workflow-Xu/funcx-ptycho-wf-{}.log".format(dataset_name)
logging.basicConfig(filename=log_file_name,
                    filemode='a',
                    format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')

while True:
    
    ### Start of folder generation ###
    src_wf_root_path = '/mnt/micdata2/velociprobe/2021-2/Xu'
    src_input_folder_prefix = "ptycho-test"
    src_input_pos_folder_prefix = "positions"
    src_output_folder_prefix = "wf-recons-test-i100"

    dest_wf_root_path = '/grand/hp-ptycho/bicer/20210723_workflow-Xu'
    dest_input_folder_prefix = "input"
    dest_output_folder_prefix = "output"

    add_larger_than = 999
    
    # Get the activated iids
    activated_iids_rows = read_activated_iids()
    activated_iids = []
    for r in activated_iids_rows:
        activated_iids.append(r[0])

    # Get the remote folders at source input folder
    src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*"
    print(src_input_folder_paths_regex)
    rid = fxc.run(src_input_folder_paths_regex,
                  endpoint_id=prisma_fx_endpoint,
                  function_id=fx_func_get_folder_paths_uuid)
    while True:
        try:
            time.sleep(1)
            src_input_folder_paths = fxc.get_result(rid)
            print(src_input_folder_paths)
        except KeyboardInterrupt:
            break
        except:
            print("Result is not ready:{}".format(sys.exc_info()[0]))
            continue
        break

    iids = []
    for src_input_folder_path in src_input_folder_paths:
        #print(src_input_folder_path)
        iid = re.findall(r'\d+', src_input_folder_path)
        if int(iid[-1]) > add_larger_than: 
            #print("id is being added:{}".format(int(iid[-1])))
            iids.append(iid[-1])
        else: continue


    # Get the remote folders at source output folder
    src_output_folder_paths_regex = f"{src_wf_root_path}/{src_output_folder_prefix}/*"
    rid = fxc.run(src_output_folder_paths_regex,
                  endpoint_id=prisma_fx_endpoint,
                  function_id=fx_func_get_folder_paths_uuid)
    while True:
        try:
            time.sleep(1)
            ex_src_output_folder_paths = fxc.get_result(rid)
            #print(ex_src_output_folder_paths)
        except KeyboardInterrupt:
            break
        except:
            print("Result is not ready:{}".format(sys.exc_info()[0]))
            continue
        break

    oiids = []
    for ex_src_output_folder_path in ex_src_output_folder_paths:
        oiid = re.findall(r'\d+', ex_src_output_folder_path)
        if int(oiid[-1]) > add_larger_than: 
            #print("id is being added:{}".format(int(oiid[-1])))
            oiids.append(oiid[-1])
        else: continue

    # Find the iids that need to be processed
    diff_iids = []
    for iid in iids:
        if (int(iid)>add_larger_than) and (iid not in oiids) and (iid not in activated_iids): 
            diff_iids.append(iid) # process only these
    
    # If there is no file to be processed sleep 1 second and recheck 
    if(len(diff_iids)==0):
        print("There is no folder to be processed, sleeping 1 secs.")
        time.sleep(1)
        continue
    #print("diff_iids:{}\niids:{}\noiids:{}\nactivated_iids:{}".format(diff_iids, iids, oiids,activated_iids))
    
    
    #print("Creating works for these ids: {}\n".format(diff_iids))
    fsrc_input_folder_paths = []
    src_input_pos_files = []
    src_output_folder_paths = []
    dest_output_folder_paths = []
    dest_input_folder_paths = []
    dest_input_pos_files = []
    iids = []
    for src_input_folder_path in src_input_folder_paths[:-1]:
        #print(src_input_folder_path)
        iid = re.findall(r'\d+', src_input_folder_path)
        #if (int(iid[-1]) <= add_larger_than) or (iid[-1] in oiids): continue
        # Skip if iid is already activated or processed
        if (iid[-1] not in diff_iids): continue
        iids.append(iid[-1])
        fsrc_input_folder_paths.append(src_input_folder_path)
        src_input_pos_files.append(f"{src_wf_root_path}/{src_input_pos_folder_prefix}/fly{iid[-1]}_0.txt")
        src_output_folder_path = f"{src_wf_root_path}/{src_output_folder_prefix}/{iid[-1]}"
        src_output_folder_paths.append(src_output_folder_path)
        dest_input_folder_path = f"{dest_wf_root_path}/{dest_input_folder_prefix}/{iid[-1]}"
        dest_input_folder_paths.append(dest_input_folder_path)
        dest_input_pos_files.append(f"{dest_input_folder_path}/fly{iid[-1]}_0.txt")
        dest_output_folder_path = f"{dest_wf_root_path}/{dest_output_folder_prefix}/{iid[-1]}"
        dest_output_folder_paths.append(dest_output_folder_path)
    src_input_folder_paths = fsrc_input_folder_paths
    #print(len(src_input_folder_paths))
    # src_input_folder_paths: diffraction patterh files to be processed @ APS
    # src_output_folder_paths: folders for reconstrcuted images after processing @ APS
    # dest_input_folder_paths: diffraction patterh files to be processed @ ALCF
    # dest_output_folder_paths: folders for reconstrcuted images after processing @ ALCF

    iids.reverse()
    src_input_folder_paths.reverse()
    src_input_pos_files.reverse()
    src_output_folder_paths.reverse()
    dest_input_folder_paths.reverse()
    dest_input_pos_files.reverse()
    dest_output_folder_paths.reverse()

    for (iid, src_input_folder_path, src_input_pos_file, src_output_folder_path, dest_input_folder_path, dest_input_pos_file, dest_output_folder_path ) in zip(iids, src_input_folder_paths, src_input_pos_files, src_output_folder_paths, dest_input_folder_paths, dest_input_pos_files, dest_output_folder_paths):
        print(f"{iid}: Source input folder: {src_input_folder_path}; Source position file: {src_input_pos_file}; Source output folder: {src_output_folder_path}")
        print(f"Dest. input folder: {dest_input_folder_path}; Dest. position file: {dest_input_pos_file}; Dest. output folder: {dest_output_folder_path}")
        print()
    
    if(len(iids)>0):
        print("diff_iids:{}\niids:{}\noiids:{}".format(diff_iids, iids, oiids))
    ### End of folder generation ###
    
      
    ### Ptycho recon params setting up ###
    script_path = '/home/bicer/projects/tike/scripts/tike-pinned-ptycho-wf.py'
    #python_path = "/home/bicer/projects/tyler/bin/python"
    python_path = "/home/bicer/miniconda3/envs/ptycho/bin/python"

    probe_path = '/grand/hp-ptycho/bicer/20210723_workflow-Xu/probes/velociprobe-probe.npy'

    rec_alg = 'lstsq_grad'
    rec_nmodes = 1
    rec_upd_pos = False
    rec_niter = 100
    rec_output_freq = 20
    rec_recover_psi = True
    rec_recover_probe= True
    rec_recover_positions = False
    rec_model = 'gaussian'
    rec_ngpu = 1
    rec_use_mpi = False
    rec_overwrite = True
    rec_auto_pin = True

    nworkers_per_node = 8

    flow_inputs = []
    gcounter = 0
    for (iid, src_input_folder_path, src_input_pos_file, src_output_folder_path,
         dest_input_folder_path, dest_input_pos_file, dest_output_folder_path ) in zip(iids,
        src_input_folder_paths, src_input_pos_files, src_output_folder_paths,
        dest_input_folder_paths, dest_input_pos_files, dest_output_folder_paths):

        rec_gpu_id = gcounter%nworkers_per_node

        flow_input = {
            "iid" : iid,
            "input": {
                "source_endpoint": f"{src_endpoint}",
                "source_path": f"{src_input_folder_path}/",
                "source_pos_path": f"{src_input_pos_file}",
                "dest_endpoint": dest_endpoint,
                "dest_path": f"{dest_input_folder_path}/",
                "dest_pos_path": dest_input_pos_file,

                "result_path": f"{dest_output_folder_path}",
                "source_result_path": f"{src_output_folder_path}",
                "fx_ep": f"{theta_fx_endpoint}",
                "fx_id": f"{func_ptycho_uuid}",
                "params": {'ifpath': f"{dest_input_folder_path}/fly{iid}_master.h5",
                           'ofpath': f"{dest_output_folder_path}/",
                           'position_path': dest_input_pos_file,
                           'probe_path': probe_path,
                           'script_path': script_path,
                           'python_path': python_path,
                           'rec_alg': rec_alg,
                           'rec_nmodes': rec_nmodes,
                           'rec_upd_pos': rec_upd_pos,
                           'rec_niter': rec_niter,
                           'rec_output_freq': rec_output_freq,
                           'rec_recover_psi': rec_recover_psi,
                           'rec_recover_probe': rec_recover_probe,
                           'rec_recover_positions': rec_recover_positions,
                           'rec_model': rec_model,
                           'rec_ngpu': rec_ngpu,
                           'rec_use_mpi': rec_use_mpi,
                           'rec_overwrite': rec_overwrite,
                           'rec_auto_pin': rec_auto_pin,
                           'rec_gpu_id':rec_gpu_id,
                           'dataset_name': dataset_name,
                           'wid':gcounter}
            }
        }
        gcounter=gcounter+1
        flow_inputs.append(flow_input)
        ### End of parameter setup
    
    for finputs in flow_inputs:
        print("Flow input iid: {}".format(finputs['iid']))
    
    print("Number of flows to be generated: {}".format(len(flow_inputs)))
    
    ### Initiate Flow ###
    nnodes = 2
    nworkers_per_node = 8

    nflows = len(flow_inputs)

    for i in range(nflows):
        flow_action = flows_client.run_flow(flow_id, flow_scope, flow_inputs[i])
        q1.append(flow_action)
        append_activated_iids([flow_inputs[i]['iid']])
        lstr=f"Flow {i} initiated and added to q1: {flow_action['action_id']}"
        logging.info(lstr)
        print(lstr)

In [ ]:
# Cancel all flows
for flow_action in q1:
    flows_client.flow_action_cancel(flow_id, flow_scope, flow_action['action_id'])